In [ ]:
import control
import plotly.express as px
import numpy as np
import pandas as pd

In [ ]:
# First order dynamical system with analytical solution, for simple systems where we can get the solution 

In [ ]:
K = 30 # Gain
T = 4 # time constant

t_start = 0
t_stop = 30
dt = 0.1

In [ ]:
t = np.arange(t_start,t_stop, dt)

In [ ]:
y = K * (1 - np.exp(-t/T))

In [ ]:
df_results = pd.DataFrame()
df_results['time'] = t
df_results['y'] = y
fig = px.line(df_results, x="time", y="y", title='1st order dynamical system analytical')
fig.show()

In [ ]:
# Using the scipy solver for the ode

In [ ]:
from scipy.integrate import odeint
# Initialization 
K = 3 
T = 4
u = 1
t_start =0
t_stop = 25
y0=0
t_odeint = np.arange(t_start,t_stop+1, dt)

def sys1order(y,t, K, T, u):
    y_dot = (1/T) * (-y + K * u)
    return y_dot

y_odeint = odeint ( sys1order, y0, t_odeint, args = (K,T,u))

In [ ]:
df_results = pd.DataFrame()
df_results['time'] = t_odeint
df_results['y'] = y_odeint
fig = px.line(df_results, x="time", y="y", title='1st order dynamical system ode')
fig.show()

In [ ]:
# Discreatization 
#y_dot ~= (y[k+1]-y[k])/Ts

In [ ]:
a = -1/T
b = K/T

uk = 1
yk = 0
Ts = dt # sample time
N = int(t_stop/Ts)
data = []
data.append(yk)

#Newton euler
for k in range(0,N):
    y_dot = sys1order(yk, t, K, T, uk)
    yk1 = yk + y_dot * Ts
    yk = yk1
    data.append(yk)
t = np.arange(t_start,t_stop+Ts, Ts)


In [ ]:
df_results = pd.DataFrame()
df_results['time'] = t
df_results['y'] = data
fig = px.line(df_results, x="time", y="y", title='1st order dynamical system Discreatization')
fig.show()

In [ ]:
#Transfer function 

In [ ]:
K = 3
T = 4
num = np.array([K])
den =np.array([T, 1])
H = control.tf(num , den)
print(H)

In [ ]:
t, y = control.step_response(H)

In [ ]:
df_results = pd.DataFrame()
df_results['time'] = t
df_results['y'] = y
fig = px.line(df_results, x="time", y="y", title='1st order dynamical system Transfer function')
fig.show()

In [ ]:
#State space model

In [ ]:
import scipy.signal as sig

#Simulation Parameters
x0 =[0,0]
t_start = 0
t_end = 30
step=1
t = np.arange(t_start, t_end, step)

K = 3
T = 4

# State space model

A = [[-1/T, 0],
     [0, 0]]
B =[[K/T],
    [0]]
C = [[1, 0]]
D = 0

sys = sig.StateSpace(A, B, C, D)

#
t, y_statespace = sig.step(sys, x0, t)

# Transfer function from state space
H = sys.to_tf()

t, y_tf = sig.step(H, x0, t)

In [ ]:
df_results = pd.DataFrame()
df_results['time'] = t
df_results['y_statespace'] = y_statespace
df_results['y_tf'] = y_tf

fig = px.line(df_results, x="time", y=['y_statespace','y_tf'], title='1st order dynamical system State Space')
fig.show()

In [ ]:
# Bode Plot Scipy lib
from plotting import bode 
# Define Transfer Function
num1 = np.array([3])
num2 = np.array([2, 1])
num = np.convolve(num1, num2)
den1 = np.array([3, 1])
den2 = np.array([5, 1])
den = np.convolve(den1, den2)
H = sig.TransferFunction(num, den)
print ('H(s) =', H)
# Frequencies
w_start = 0.01
w_stop = 10
step = 0.01
N = int ((w_stop-w_start )/step) + 1
w = np.linspace (w_start , w_stop , N)
# Bode Plot
w, mag, phase = sig.bode(H, w)
bode_plot = bode(w, mag, phase)
bode_plot.show()

In [ ]:
H = control.tf(num, den)
print ('H(s) =', H)

w, mag, phase  =control.bode(H, dB=True)

In [ ]:
# PID control - discrete

In [ ]:
a = -1/T
b = K/T


Ts = dt # sample time
N = int(t_stop/Ts)

#Propotional and integral gainss
Kp = 0.5
Ti = 5

# Reference signal
r = 5

y = np.zeros(N+2)
e = np.zeros(N+2)
u = np.zeros(N+2)
e_int = 0
#Newton euler
t = np.arange(t_start,t_stop+2*Ts, Ts)

for k in range(N+1):
    # Error
    e[k] = r - y[k]
    # Integrated error
    e_int +=e[k] 
    #PI controller
    u[k] = Kp*e[k] + Kp/Ti*e_int
    # Dynamical system
    y_dot = sys1order(y[k], t[k], K, T, u[k])
    # State progression
    y[k+1] = y[k] + y_dot * Ts



In [ ]:
df_results = pd.DataFrame()
df_results['time'] = t
df_results['e'] = e
df_results['u'] = u
df_results['y'] = y

fig = px.line(df_results, x="time", y=['e','u','y'], title='1st order dynamical system with PI control')
fig.show()

In [ ]:
# Stability Analysis
import matplotlib.pyplot as plt
# Transfer Function Process
K = 3; T = 4
num_p = np.array ([K])
den_p = np.array ([T , 1])
Hp = control.tf(num_p , den_p)
print ('Hp(s) =', Hp)
# Transfer Function PI Controller
Kp = 0.4
Ti = 2
num_c = np.array ([Kp*Ti, Kp])
den_c = np.array ([Ti , 0])
Hc = control.tf(num_c, den_c)
print ('Hc(s) =', Hc)
# Transfer Function Measurement
Tm = 1
num_m = np.array ([1])
den_m = np.array ([Tm , 1])
Hm = control.tf(num_m , den_m)
print ('Hm(s) =', Hm)
# Transfer Function Lowpass Filter
Tf = 1
num_f = np.array ([1])
den_f = np.array ([Tf , 1])
Hf = control.tf(num_f , den_f)
print ('Hf(s) =', Hf)
# The Loop Transfer function
L = control.series(Hc, Hp, Hf, Hm)

In [ ]:
# Tracking transfer function
T = control.feedback(L,1)
print ('T(s) =', T)
# Step Response Feedback System (Tracking System)
t, y = control.step_response(T)
plt.figure(1)
plt.plot(t,y)
plt.title("Step Response Feedback System T(s)")
plt.grid()


In [ ]:
# Bode Diagram with Stability Margins
plt.figure(2)
control.bode(L, dB=True, deg=True, margins=True)
# Poles and Zeros
plt.figure(3)
control.pzmap(T)
p = control.pole(T)
z = control.zero(T)
print("poles = ", p)
# Calculating stability margins and crossover frequencies
gm , pm , w180 , wc = control.margin(L)
# Convert gm to Decibel
gmdb = 20 * np.log10(gm)
print("wc =", f'{wc:.2f}', "rad/s")
print("w180 =", f'{w180:.2f}', "rad/s")
print("GM =", f'{gm:.2f}')
print("GM =", f'{gmdb:.2f}', "dB")
print("PM =", f'{pm:.2f}', "deg")
# Find when Sysem is Marginally Stable (Kritical Gain - Kc)
Kc = Kp*gm
print("Kc =", f'{Kc:.2f}')